In [1]:
import pandas as pd

df = pd.read_stata("./serrano/serrano_2024_Stata/serrano.dta")


['ORGNR', 'KOMORGNR', 'KOMNAMN', 'BSTYP', 'STATUS', 'REDTYP', 'BELKOD', 'UTDBEL', 'NTOMS', 'LAGERF', 'AKTARB', 'ROINTOV1', 'RAVAR', 'HANDVAR', 'EXTKOSOV', 'PERSKOS', 'AVSKRIV', 'JFRST1', 'RORKOOV1', 'RORRESUL', 'RESAND', 'RTEINKNC', 'RTEINEXT', 'RTEINOV', 'RTEKOKNC', 'RTEKOEXT', 'RTEKOOV', 'JFRSTFIN', 'RESEFIN', 'EXTRAINT', 'EXTRAKOS', 'KNCBDR', 'AGTSK', 'BSLDISP', 'SKATTER', 'MININTRR', 'RESAR', 'KOSALVAR', 'BRUTORES', 'FORSKO', 'ADMKO', 'FOUKO', 'JFRST2', 'ROINTOV2', 'RORKOOV2', 'EJINBET', 'FOUBAUTG', 'PATLIC', 'GOODWILL', 'IMANLOV', 'IMANLSU', 'BYGGMARK', 'MASK', 'INVENT', 'MASKINV', 'MATANLOV', 'MATANLSU', 'ANDKNC', 'LFORDKNC', 'LANDELAG', 'FIANLTOV', 'FIANLTSU', 'ANLTSU', 'PAGARB', 'LAGEROV', 'LAGERSU', 'KUNDFORD', 'KFORDKNC', 'KFORDOV', 'KFORDSU', 'KPLACSU', 'KABASU', 'OMSTGSU', 'TILLGSU', 'AKTIEKAP', 'OVERKURS', 'UPPSKR', 'OVRGBKAP', 'BALRES', 'KNCBDREL', 'AGTSKEL', 'RESARB', 'EKSU', 'OBESKRES', 'MININTR', 'AVSSU', 'LSKKRIN', 'LSKKNC', 'LSKOV', 'LSKSU', 'KSKKRIN', 'KSKLEV', 'KSKKNC', 'KSKOV', 'KSKSU', 'EKSKSU', 'RTENTO', 'ANTANST', 'LONLEDN', 'LONOV', 'SOCKOSTN', 'TANTLEDN', 'RESLONOV', 'AVGVED', 'AVSKSALV', 'AVSKFSG', 'AVSKADM', 'AVSKFOU', 'AVSKOV2', 'AVSKOSPC', 'INTFTG', 'INTFAST', 'SAKOV', 'SAKKOM', 'SAKSU', 'AGTSKV', 'ANSVFOV', 'ANSVFKOM', 'ANSVFSU', 'CHKRBEV', 'CHKRUTN', 'REVBER', 'BOLSTPRO', 'MODDTM', 'BSLSTART', 'BSLSLUT']

In [ ]:
# Filter the DataFrame
filtered_df = df[(df['ser_year'] >= 2007) & # remove data before 2007
                 (df['ser_jurform'] == 49) & # aktiebolag
                 (df['ser_aktiv'] == 1) & # active companies
                 (df['ser_ftgkategori'] == 30)] # private companies, i.e., not state-owned etc.



# Calculate the annual growth rate for each company and add it as a new column
filtered_df['TURNOVER_GROWTH'] = filtered_df.groupby('ORGNR')['rr01_ntoms'].pct_change()

# Identify high-growth periods and create a new binary column
filtered_df['HIGH_GROWTH'] = filtered_df.groupby('ORGNR').apply(
    lambda x: ((x['TURNOVER_GROWTH'] > 0.20).rolling(window=3).sum() == 3) & 
              (x['ser_stklf'].iloc[0] in [3, 4, 5, 6, 7]) # only consider companies with at least 10 employees, based on size category
).reset_index(level=0, drop=True).astype(int)



/var/folders/pd/d0m27jy17d36rd2w_8y762cr0000gn/T/ipykernel_57696/2909734736.py:10: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  filtered_df['TURNOVER_GROWTH'] = filtered_df.groupby('ORGNR')['rr01_ntoms'].pct_change()
/var/folders/pd/d0m27jy17d36rd2w_8y762cr0000gn/T/ipykernel_57696/2909734736.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  filtered_df['TURNOVER_GROWTH'] = filtered_df.groupby('ORGNR')['rr01_ntoms'].pct_change()
/var/folders/pd/d0m27jy17d36rd2w_8y762cr0000gn/T/ipykernel_57696/2909734736.py:10: SettingWithCopyWarning: 
A value is tryi

In [16]:
# Drop rows where ser_year is 2007
# it is there in in the first place in order to calculate the growth rate for 2008.
filtered_df = filtered_df[filtered_df['ser_year'] != 2007]

In [17]:
filtered_df['ORGNR'].nunique() # amount of companies in this filtered dataset.

855819

In [20]:
# Filter to keep only companies that have at least one record with HIGH_GROWTH == 1
high_growth_orgnr = filtered_df[filtered_df['HIGH_GROWTH'] == 1]['ORGNR'].unique()
filtered_high_growth_df = filtered_df[filtered_df['ORGNR'].isin(high_growth_orgnr)]

In [29]:
filtered_high_growth_df['ORGNR'].nunique() # amount of companies in this filtered dataset of HGFs.

3217

In [18]:
# Save the filtered DataFrame to an Excel file
output_path = "./serrano/serrano_2024_Stata/firms.xlsx"
filtered_df.head(20000).to_excel(output_path, index=False)

print(f"Filtered data saved to {output_path}")

Filtered data saved to ./serrano/serrano_2024_Stata/firms.xlsx


In [22]:
# Save the filtered DataFrame to an Excel file
output_path = "./serrano/serrano_2024_Stata/hgfs.xlsx"
filtered_high_growth_df.head(20000).to_excel(output_path, index=False)

print(f"Filtered data saved to {output_path}")

Filtered data saved to ./serrano/serrano_2024_Stata/hgfs.xlsx


In [27]:
# This cell is about understanding the industries of the high growth companies.
# work in progress

# Filter to keep only the rows where HIGH_GROWTH is 1
high_growth_years = filtered_high_growth_df[filtered_high_growth_df['HIGH_GROWTH'] == 1]

# Sort by ORGNR and ser_year to ensure we get the latest year for each company
high_growth_years_sorted = high_growth_years.sort_values(by=['ORGNR', 'ser_year'], ascending=[True, False])

# Drop duplicates to keep only the latest year for each company
latest_high_growth_years = high_growth_years_sorted.drop_duplicates(subset='ORGNR', keep='first')

# Extract the relevant columns
high_growth_industries = latest_high_growth_years[['ORGNR', 'ser_year', 'bransch_sni071']]

high_growth_industries

,ORGNR,ser_year,bransch_sni071
417452,5.560013e+09,2010.0,58131.0
417871,5.560015e+09,2017.0,1110.0
419089,5.560021e+09,2019.0,70100.0
424463,5.560061e+09,2021.0,46769.0
425381,5.560068e+09,2018.0,70100.0
...,...,...,...
11700525,5.592295e+09,2022.0,70220.0
11707101,5.592312e+09,2022.0,49410.0
11708087,5.592315e+09,2022.0,81210.0
11711873,5.592324e+09,2022.0,52290.0


In [34]:
high_growth_industries.head(50)

,ORGNR,ser_year,bransch_sni071
417452,5.560013e+09,2010.0,58131.0
417871,5.560015e+09,2017.0,1110.0
419089,5.560021e+09,2019.0,70100.0
424463,5.560061e+09,2021.0,46769.0
425381,5.560068e+09,2018.0,70100.0
429060,5.560096e+09,2014.0,24530.0
429384,5.560098e+09,2017.0,68201.0
431267,5.560111e+09,2020.0,46762.0
431787,5.560113e+09,2018.0,41200.0
437766,5.560147e+09,2014.0,45310.0
